# Project Management

## Internals

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

### Param

In [2]:
_pathTasks='tasks.csv'
_pathAvail='avail.csv'
_ProjectStartDate = dt.datetime(year=2016,month=11,day=21)

### Load data

In [3]:
_TaskHeaders = ['TaskId','Name','WorkLoad','Who','StartDate','EndDate','Priority','PreviousTaskId']

def LoadTask(path):
        data = pd.read_csv(path, header=0,names=_TaskHeaders, dtype={'PreviousTaskId':object})
        data.StartDate = pd.to_datetime(data.StartDate)
        data.EndDate = pd.to_datetime(data.EndDate)
        data.index=data.TaskId
        return data
def LoadAvailability(path):
    '''
    This function load an availability matrix
    '''
    availCSV = pd.read_csv(path)
    availCSV.Date = pd.to_datetime(availCSV.Date)
    return availCSV
    

### Save

In [4]:
def SaveData(data,path):
    data.to_csv(path)

### Domain
* Find tasks of a personn
* Order tasks by priority then ID (just in case)
* Compute end dates
    * Find previous task of person and previous task id
    * Based on the max start date and workload, iterate to see the day when the task will be finised


In [5]:
def FindPreviousTaskOfSomeone(tasks, current):
    '''
    Find the previous task in order of doing
    Expects an order tasks pandas by priority desc and taskid 
    '''
    whosTasks = FindTasksOfSomeone(tasks,current.Who)    
    prevTasks= whosTasks.loc[:current.TaskId]
    if len(prevTasks)<2 :
        return None
    else :
        return prevTasks.iloc[-2]
    
    

In [6]:
def FindTasksOfSomeone(tasks, who):
    return tasks[tasks.Who==who]
def FindTaskIdById(tasks, taskid):
    return tasks.loc[taskid]
def OrderTaskByPriority(tasks):
    return tasks.sort_values(by=['Priority','TaskId'], ascending=[False,True])

In [7]:
def InitEmptyOccupation(avail,startDate, endDate):
    users = avail.Who.unique()
    duration = (endDate-startDate).days
    return pd.DataFrame(data=np.zeros(shape=(duration,len(users)), dtype=np.int32),\
                        columns=users,\
                        index=np.arange(startDate,endDate, dtype='datetime64[D]'))

In [27]:
def ComputeEndDate(availability, task):
    '''
    Compute the end date based on the occupation of someone, a start date and a duration
    '''
    availWho = availability[(availability.Who==task.Who) & (availability.Date>=task.StartDate)]  
    currentOccupation = 0
    endDate=None
    for index,item in availWho.iterrows():
        currentOccupation=currentOccupation+item.Available
        print item
        print currentOccupation
        if(currentOccupation==task.WorkLoad):
            endDate=item.Date
            break;
    if(endDate==None):
        raise Exception('Task cannot be ended',task)
    return endDate
        

In [32]:
def SetOccupiedBy(occupation,task):
    occupation.loc[task.StartDate:task.EndDate,task.Who]=task.TaskId

In [13]:
def ComputeOccupation(avail,tasks):
    '''
    '''
    #For everyone, iterate on tasks and compute them
    for who in tasks.Who.unique():
        print who

In [34]:
_tasks.loc[1,'StartDate']=dt.datetime(2016,11,23)
_tasks.loc[1,'EndDate']=dt.datetime(2016,11,25)
SetOccupiedBy(_occupation,_tasks.loc[1])

In [35]:
_occupation

,CGA
2016-11-21,0
2016-11-22,0
2016-11-23,1
2016-11-24,1
2016-11-25,1
2016-11-26,0
2016-11-27,0
2016-11-28,0
2016-11-29,0
2016-11-30,0


## Load Data

In [9]:
_tasks = LoadTask(_pathTasks)

In [29]:
_avail = LoadAvailability(_pathAvail)

In [11]:
_tasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority,PreviousTaskId
TaskId,,,,,,,,
1,1,Test 1,4,CGA,NaT,NaT,150,NaN
2,2,Test 2,4,CGA,NaT,NaT,130,3
3,3,Test 3,4,ACH,NaT,NaT,140,NaN


In [12]:
_avail.head(5)

,Who,Date,Available
0,CGA,2016-11-21,1
1,CGA,2016-11-22,1
2,CGA,2016-11-23,1
3,CGA,2016-11-24,1
4,CGA,2016-11-25,1


NaT
Empty DataFrame
Columns: [Who, Date, Available]
Index: []


[[ 0.  0.  0.]
 [ 0.  0.  0.]]


In [33]:
_occupation = InitEmptyOccupation(_avail,_ProjectStartDate,_ProjectStartDate+dt.timedelta(365))

In [44]:
len(_avail.Who.unique())

1

In [27]:
help (np.zeros)

Help on built-in function zeros in module numpy.core.multiarray:

zeros(...)
    zeros(shape, dtype=float, order='C')
    
    Return a new array of given shape and type, filled with zeros.
    
    Parameters
    ----------
    shape : int or sequence of ints
        Shape of the new array, e.g., ``(2, 3)`` or ``2``.
    dtype : data-type, optional
        The desired data-type for the array, e.g., `numpy.int8`.  Default is
        `numpy.float64`.
    order : {'C', 'F'}, optional
        Whether to store multidimensional data in C- or Fortran-contiguous
        (row- or column-wise) order in memory.
    
    Returns
    -------
    out : ndarray
        Array of zeros with the given shape, dtype, and order.
    
    See Also
    --------
    zeros_like : Return an array of zeros with shape and type of input.
    ones_like : Return an array of ones with shape and type of input.
    empty_like : Return an empty array with shape and type of input.
    ones : Return a new array setting v